In [1]:
try:
    sc and spark
except NameError as e:
    import pyspark
    import pyspark.sql
    
    sc = pyspark.SparkContext()
    spark = pyspark.sql.SparkSession.builder.appName("test").getOrCreate()

In [2]:
import graphframes as gf
from pyspark.sql.functions import col
from functools import reduce


In [3]:
# READ DATA
records = spark.read.option("header","true").csv("data/test_records.csv")

In [4]:
records.first()

Row(serial_no='FVH6107', hostname='UK-OUT-1383', mac='21:35:ee:fe:1d:03', ip=None, dns_name='UK-OUT-1383.msn.com', distinguished_name=None, device_type='Server', os_type=None, os='Ubuntu Linux', os_version=None, os_service_pack=None, user=None, last_logon_timestamp='2018-08-08 07:01:39', last_pw_change_timestamp='2016-01-28 07:01:39 UTC', owner='Krystyna Garrold', first_seen_timestamp='2017-04-15 09:40:59', last_seen_timestamp='2021-02-16 07:01:39 UTC', last_scan_timestamp=None, source_type='database', source_name='LanDesk', record_id='0')

In [21]:
mirrorColNames = [f"_{col}" for col in records.columns]
mirror = records.toDF(*mirrorColNames)

In [22]:
# generate edges
r=records
m=mirror
match_cols=["ip", "mac", "hostname"]
mcond=[col(c) == col(f'_{c}') for c in match_cols]
cond = [(col("record_id") != col("_record_id")) & \
        reduce(lambda x,y: x | y, mcond)]
cond

# 

edges = r.join(m, cond).select("record_id", "_record_id")
# r["record_id"].notEqual(m["_record_id"]) )
# & r("ip").equalTo(m("_ip")) )

In [25]:
cond

[Column<b'((NOT (record_id = _record_id)) AND (((ip = _ip) OR (mac = _mac)) OR (hostname = _hostname)))'>]

In [23]:
# generate nodes
nodes=records.withColumnRenamed('record_id', 'id')

# generate edge list
edge_list=edges.withColumnRenamed('record_id', 'src').withColumnRenamed('_record_id', 'dst')

# create graph
graph = gf.GraphFrame(nodes, edge_list)

In [24]:
# run connected components
spark.sparkContext.setCheckpointDir("/tmp/checkpoints")
cc = graph.connectedComponents()

In [68]:
cc.show()

+--------------+-----------------+--------------------+----------+----------------+-----------+---+-------------+
|            ip|              mac|            hostname| serial_no|           owner|source_name| id|    component|
+--------------+-----------------+--------------------+----------+----------------+-----------+---+-------------+
|          null|29:0b:58:3d:26:6d|isoulsby3v@behanc...|KXHGN8D8KM|    Izzy Soulsby|         LD|  0| 300647710720|
|          null|a9:11:e3:35:23:fa| cpollak74@nifty.com|Y5K30C65M3| Cathlene Pollak|         LD|  1|1236950581248|
|          null|90:22:03:25:11:d8|  ckissock53@php.net|7ZGA85QA5H|   Colin Kissock|         LD|  2|1623497637888|
|          null|4c:31:07:e9:9a:90|  bdruhan1m@1688.com|U0UOS9R43J|Brittaney Druhan|         LD|  3| 154618822656|
|106.60.189.138|             null|  bdruhan1m@1688.com|      null|            null|         SY|  4| 154618822656|
|106.60.189.138|4c:31:07:e9:9a:90|                null|      null|            null|     

In [6]:
edges.show()

+---------+----------+
|record_id|_record_id|
+---------+----------+
|        3|         4|
|        3|         5|
|        4|         3|
|        4|         5|
|        5|         3|
|        5|         4|
+---------+----------+



In [15]:
for r in edges.groupBy('record_id').count().collect():
    print(r)

Row(record_id='3', count=2)
Row(record_id='5', count=2)
Row(record_id='4', count=2)


In [17]:
import sys
sys.path

['/home/jovyan/work',
 '/tmp/spark-7b71b98f-9773-4c6c-8100-1a4e09254dd6/userFiles-879341e2-dbe2-47ab-9d10-9b7e5fc5424f',
 '/usr/local/spark/python',
 '/usr/local/spark/python/lib/py4j-0.10.7-src.zip',
 '/opt/conda/lib/python37.zip',
 '/opt/conda/lib/python3.7',
 '/opt/conda/lib/python3.7/lib-dynload',
 '',
 '/opt/conda/lib/python3.7/site-packages',
 '/opt/conda/lib/python3.7/site-packages/IPython/extensions',
 '/home/jovyan/.ipython']

In [16]:
from pyspark.sql.functions import monotonically_increasing_id
records.withColumn("id", monotonically_increasing_id())

In [17]:
gg.select(["id", "ip", "record_id"]).show()

+---+----+---------+
| id|  ip|record_id|
+---+----+---------+
|  0|null|        0|
|  1|null|        1|
|  2|null|        2|
|  3|null|        3|
|  4|null|        4|
|  5|null|        5|
|  6|null|        6|
|  7|null|        7|
|  8|null|        8|
|  9|null|        9|
| 10|null|       10|
| 11|null|       11|
| 12|null|       12|
| 13|null|       13|
| 14|null|       14|
| 15|null|       15|
| 16|null|       16|
| 17|null|       17|
| 18|null|       18|
| 19|null|       19|
+---+----+---------+
only showing top 20 rows

